# Comparing a USGSCSM and ISIS camera for Dejittered HiRISE image

In [1]:
import pyproj
import pvl
import numpy as np
import os
import pandas as pd
import json
import knoten
import csmapi

os.environ['ISISROOT'] = '/usgs/pkgs/isis3.8.0_RC1/install'
from pysis import isis
from pysis.exceptions import ProcessError

## Update the ISIS cube
Requires ESP_016076_2175_REDmos_hijitreged.balance.noproj.8bit.cub in data directory

We also have to set the shapemodel to the ellipsoid because CSM does not natively support DEM intersections

In [2]:
cub_loc = 'data/ESP_016076_2175_REDmos_hijitreged.balance.noproj.8bit.cub'
try:
    isis.editlab(from_=cub_loc, grpname='Kernels', Keyword='ShapeModel', value='Null')
except ProcessError as e:
        print(f'Failed to set shapemodel for {cub_loc} to ellipsoid:\n{e.stderr}')

## Make a CSM sensor model

In [3]:
camera = knoten.csm.create_csm(cub_loc)

## Get the total number of lines / samples

In [4]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

## Define functions that compare ISIS and USGSCSM pixels

In [5]:
def check_pixel_isis_ground(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    coord = csmapi.EcefCoord(*bodyfixed)
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

In [6]:
def check_pixel_csm_ground(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using CSM, then projects
    the result back into an image coordinate using ISIS and computes the difference
    between image coordinates.
    
    """
    # Create an image coordinate based on line and sample
    image_coord = csmapi.ImageCoord(line, sample)
    
    # Project image coordinate to a ground point
    out = camera.imageToGround(image_coord, 0.0)
    
    # Results of imageToGround are ECEF, campt requires lat/lon
    a_rad, b_rad = knoten.csm.get_radii(camera)
    lon,lat,_ = ecef_to_lla(out.x, out.y, out.z, a_rad, b_rad)

    # campt requires positive east 360
    lon = (lon+360)%360
    
    try:
        pvl_output = isis.campt(from_=cub_loc, type='ground', latitude=lat, longitude=lon, allowoutside=True)
        output = pvl.loads(pvl_output)
        isis_line = output['GroundPoint']['Line']
        isis_sample = output['GroundPoint']['Sample']
        # (.5,.5) in CSM == (1,1) in ISIS, so we have to add (.5,.5) to the CSM pixels
        line_diff = line - isis_line +.5
        sample_diff = sample - isis_sample +.5
    except ProcessError as e:
        print(f'({line}, {sample}): {e.stderr}')
        line_diff = np.NaN
        sample_diff = np.NaN
        
    return line_diff, sample_diff
    

In [7]:
def ecef_to_lla(x, y, z, a_radius, b_radius):
    """ Converts from earth-centric, earth-fixed to lat, lon, altitude
    """
    ecef = pyproj.Proj(proj='geocent', a=a_radius, b=a_radius)
    lla = pyproj.Proj(proj='latlong',  a=a_radius, b=a_radius)
    
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z)
    # convert planetographic to planetocentric
#     lat = np.degrees(np.arctan(np.tan(np.radians(lat))*(b_radius/a_radius)*(b_radius/a_radius)))
    return lon, lat, alt

## Compare top left, top right, bottom left, bottom right, and center pixels using check_pixel

In [8]:
line_mesh, sample_mesh = np.meshgrid(np.linspace(1, n_lines-0.1, 50), np.linspace(1, n_samples, 5))
line_vec = line_mesh.flatten()
sample_vec = sample_mesh.flatten()
pixels_dict = {'line' : line_vec,
               'sample' : sample_vec}

# Create a dataframe to store the results of pixel comparison
pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff_csm_ground'] = np.NaN
pixels_df['sample_diff_csm_ground'] = np.NaN
pixels_df['line_diff_isis_ground'] = np.NaN
pixels_df['sample_diff_isis_ground'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff_isis_ground'], pixels_df.iloc[idx]['sample_diff_isis_ground'] = check_pixel_isis_ground(camera, cub_loc, row['line'], row['sample'])
    pixels_df.iloc[idx]['line_diff_csm_ground'], pixels_df.iloc[idx]['sample_diff_csm_ground'] = check_pixel_csm_ground(camera, cub_loc, row['line'], row['sample'])

In [9]:
pixels_df.describe()

,line,sample,line_diff_csm_ground,sample_diff_csm_ground,line_diff_isis_ground,sample_diff_isis_ground
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,22500.450000,10000.500000,0.002572,0.000037,-0.002846,-0.000069
std,13279.100365,7084.898253,0.001695,0.000230,0.001777,0.000259
min,1.000000,1.000000,-0.000137,-0.000591,-0.007473,-0.000733
25%,11021.138776,5000.750000,0.001113,-0.000081,-0.004117,-0.000169
50%,22500.450000,10000.500000,0.002469,0.000028,-0.002765,-0.000015
75%,33979.761224,15000.250000,0.003775,0.000188,-0.001340,0.000090
max,44999.900000,20000.000000,0.006773,0.000574,0.000064,0.000447
